In [8]:
import pandas as pd
import requests
from pathlib import Path
from Bio import SeqIO
from Bio.Blast import NCBIXML

In [4]:
#Parse blast output
input_file = Path('Data/1_uniref90_blast_hits.xml')

with open(input_file) as f:
  blast_records = NCBIXML.parse(f)
  data = []

  # Iterate PSIBLAST rounds (here just one since it is a simple BLAST)
  for blast_record in blast_records:
      query_id = blast_record.query

      # Iterate alignments
      for i, alignment in enumerate(blast_record.alignments):
          subject_id = alignment.title

          # Iterate pairwise alignments
          for hsp in alignment.hsps:
              data.append((query_id,
                              subject_id,
                              blast_record.query_length,
                              hsp.query,
                              hsp.match,
                              hsp.sbjct,
                              hsp.query_start,
                              hsp.query_end,
                              hsp.sbjct_start,
                              hsp.sbjct_end,
                              hsp.identities,
                              hsp.positives,
                              hsp.gaps,
                              hsp.expect,
                              hsp.score))

              # Skip duplicated subjects
              break

df = pd.DataFrame(data, columns=["query_id", "subject_id", "query_len",
                                  "query_seq", "match_seq", "subject_seq",
                                "query_start", "query_end", "subject_start", "subject_end",
                                "identity", "positive", "gaps", "eval", "bit_score"])
df.iloc[100]

query_id                                                EMBOSS_001
subject_id       UR90:UniRef90_UPI0025969A39 integrin beta-7 is...
query_len                                                      247
query_seq        GFGSFVDKTVLPFVSTVPSKLRHPCPTRLERCQSPFSFHHVLSLTG...
match_seq        GFGSFVDKTVLPFVSTVPSKLRHPCPTRLERCQ PFSFHHVLSLTG...
subject_seq      GFGSFVDKTVLPFVSTVPSKLRHPCPTRLERCQPPFSFHHVLSLTG...
query_start                                                     46
query_end                                                      247
subject_start                                                  164
subject_end                                                    365
identity                                                       198
positive                                                       199
gaps                                                             0
eval                                                           0.0
bit_score                                                   10

In [10]:
df.iloc[999]

query_id                                                EMBOSS_001
subject_id       UR90:UniRef90_A0A672Z034 Integrin beta n=3 Tax...
query_len                                                      247
query_seq        EGYPVDLYYLMDLSYSMKDDLERVRQLGHALLVRLQEVTHSVRIGF...
match_seq        +GYPVDLYYLMDLS SMKDDL+ V+ LG  L   L+++T   +IGF...
subject_seq      QGYPVDLYYLMDLSNSMKDDLQNVKGLGKDLFAALKKITQHAQIGF...
query_start                                                      2
query_end                                                      247
subject_start                                                  122
subject_end                                                    364
identity                                                       144
positive                                                       182
gaps                                                             5
eval                                                           0.0
bit_score                                                    7

In [ ]:
# Get Uniref90 accessions
out_file = 'Data/1_uniref_accessions.list'

with open(out_file, 'w') as fout:
  accessions = list(set([acc.split()[0].split(":")[1] for acc in df['subject_id']]))
  fout.write('\n'.join(accessions) + "\n")

- Accessions were used to retrieve Uniref90 sequences
- We then aligned them with ClustalOmega, and edited the alignment in Jalview by selecting columns with conservation > 5 and trimming to the left and right of them
- We then generated the PSSM and HMM from the edited alignment, and used them to retrieve hits with PSI-Blast and HMMSEARCH against SwissProt

In [13]:
import requests
import json

In [34]:
# Collect ground truth

URL="https://www.ebi.ac.uk/interpro/api/protein/reviewed/entry/InterPro/IPR002369/"
headers={'Content-Type':'application/json'}
r=requests.get(URL,headers=headers)

In [38]:
r.json()

{'count': 46,
 'next': 'https://www.ebi.ac.uk/interpro/api/protein/reviewed/entry/InterPro/IPR002369/?cursor=source%3As%3Ap26011',
 'previous': None,
 'results': [{'metadata': {'accession': 'A2A863',
    'name': 'Integrin beta-4',
    'source_database': 'reviewed',
    'length': 1818,
    'source_organism': {'taxId': '10090',
     'scientificName': 'Mus musculus',
     'fullName': 'Mus musculus (Mouse)'}},
   'entries': [{'accession': 'IPR002369',
     'entry_protein_locations': [{'fragments': [{'start': 38,
         'end': 457,
         'dc-status': 'CONTINUOUS',
         'representative': False}],
       'model': None,
       'score': None}],
     'protein_length': 1818,
     'source_database': 'interpro',
     'entry_type': 'domain',
     'entry_integrated': None}]},
  {'metadata': {'accession': 'A5Z1X6',
    'name': 'Integrin beta-1',
    'source_database': 'reviewed',
    'length': 798,
    'source_organism': {'taxId': '9837',
     'scientificName': 'Camelus bactrianus',
     'ful

In [36]:
for node in r.json()['results']:
    print(node['entries'])

[{'accession': 'IPR002369', 'entry_protein_locations': [{'fragments': [{'start': 38, 'end': 457, 'dc-status': 'CONTINUOUS', 'representative': False}], 'model': None, 'score': None}], 'protein_length': 1818, 'source_database': 'interpro', 'entry_type': 'domain', 'entry_integrated': None}]
[{'accession': 'IPR002369', 'entry_protein_locations': [{'fragments': [{'start': 34, 'end': 464, 'dc-status': 'CONTINUOUS', 'representative': False}], 'model': None, 'score': None}], 'protein_length': 798, 'source_database': 'interpro', 'entry_type': 'domain', 'entry_integrated': None}]
[{'accession': 'IPR002369', 'entry_protein_locations': [{'fragments': [{'start': 34, 'end': 464, 'dc-status': 'CONTINUOUS', 'representative': False}], 'model': None, 'score': None}], 'protein_length': 798, 'source_database': 'interpro', 'entry_type': 'domain', 'entry_integrated': None}]
[{'accession': 'IPR002369', 'entry_protein_locations': [{'fragments': [{'start': 37, 'end': 460, 'dc-status': 'CONTINUOUS', 'representa

In [44]:
accessions=[]
names=[]
length=[]
taxon=[]
start=[]
end=[]
URL="https://www.ebi.ac.uk:443/interpro/api/protein/reviewed/entry/pfam/PF00362/"
headers={'Content-Type':'application/json'}
r=requests.get(URL,headers=headers)
for node in r.json()['results']:
        metadata=node['metadata']    
        entry=node['entries'][0]
        accessions.append(metadata['accession'])
        names.append(metadata['name'])
        length.append(metadata['length'])
        taxon.append(metadata['source_organism']['taxId'])
        start.append(entry['entry_protein_locations'][0]['fragments'][0]['start'])
        end.append(entry['entry_protein_locations'][0]['fragments'][0]['end'])
while r.json()['next']:
    URL=r.json()['next']
    r=requests.get(URL,headers=headers)
    for node in r.json()['results']:
        metadata=node['metadata']    
        entry=node['entries'][0]
        accessions.append(metadata['accession'])
        names.append(metadata['name'])
        length.append(metadata['length'])
        taxon.append(metadata['source_organism']['taxId'])
        start.append(entry['entry_protein_locations'][0]['fragments'][0]['start'])
        end.append(entry['entry_protein_locations'][0]['fragments'][0]['end'])

In [45]:
d={'accession':accessions,'name':names,'length':length,'taxon':taxon,'start':start,'end':end}
ground_truth=pd.DataFrame(data=d)

In [48]:
ground_truth.head()

,accession,name,length,taxon,start,end
0,A2A863,Integrin beta-4,1818,10090,128,370
1,A5Z1X6,Integrin beta-1,798,9837,138,382
2,B0FYY4,Integrin beta-1,798,9940,138,382
3,O54890,Integrin beta-3,787,10090,133,380
4,O70309,Integrin beta-5,798,10090,134,382


In [49]:
psiblast_hits=pd.read_csv('Data/2_psiblast_uniref90_hits.csv')

In [50]:
psiblast_hits

,Integrin,P07228.1,60.329,668,261,3,1,667,139,803,0.0,763,75.00
0,Integrin,B0FYY4.1,60.928,668,256,4,1,667,135,798,0.000,762.0,74.40
1,Integrin,P12607.1,61.078,668,256,3,1,667,134,798,0.000,759.0,75.30
2,Integrin,P12606.1,61.078,668,256,3,1,667,134,798,0.000,758.0,75.45
3,Integrin,P09055.1,60.629,668,258,4,1,667,135,798,0.000,755.0,74.40
4,Integrin,P49134.1,60.629,668,259,3,1,667,135,799,0.000,755.0,75.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,Integrin,Q8NDA2.3,13.057,314,220,11,342,655,4630,4890,0.042,41.0,19.75
380,Integrin,Q9NR61.1,16.028,287,197,12,344,623,226,475,0.042,40.7,24.39
381,Integrin,Q9QW30.1,16.988,259,185,8,330,582,301,535,0.044,41.0,24.71
382,Integrin,O35516.2,17.375,259,184,8,330,582,301,535,0.044,41.0,25.10
